In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [71]:
aoi = ee.Geometry.Polygon(
    [[
        [163.299796, -77.572265],
        [163.001417, -77.597331],
        [163.075640, -77.665095],
        [163.302008, -77.636509]
    ]]
)


filtered_aster = ee.ImageCollection("ASTER/AST_L1T_003") \
    .filterBounds(aoi)

print("Number of images in McMurdo Dry Valleys:", filtered_images.size().getInfo())

Number of images in McMurdo Dry Valleys: 299


In [72]:
def maskClouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

In [73]:
aster_collection = ee.ImageCollection('ASTER/AST_L1T_003').filterBounds(aoi) \
    .map(maskClouds)

In [74]:
# select B09, the thermal infared band
thermal_band = 'B09'

# Function to select thermal IR band and rename
def selectThermalIR(image):
    return image.select(thermal_band).rename('Thermal_IR')

In [70]:
# Apply function to select thermal IR band to filtered ASTER collection
aster_with_thermal_ir = filtered_aster.map(selectThermalIR)

EEException: Image.select: Band pattern 'B09' did not match any bands. Available bands: [B01_mean, B02_mean, B3N_mean, B04_mean, B05_mean, B06_mean, B07_mean, B08_mean, B09_mean, B10_mean, B11_mean, B12_mean, B13_mean, B14_mean, Radiance_mean]

In [68]:
task = ee.batch.Export.image.toDrive(
    image=radiance_stats,
    description='ASTER_radiance',
    crs=projection['crs'],
    crsTransform=projection['transform'],
    region=aoi,
)
task.start()


TypeError: 'ImageCollection' object is not subscriptable

In [ ]:

# Function to export image to Google Drive
def exportImageToDrive(image):
    image_name = image.id().getInfo().split('/')[-1]
    export_params = {
        'image': image,
        'description': image_name,
        'scale': 30,  # Adjust scale as needed
        'region': aoi,
        'fileFormat': 'GeoTIFF',
        'folder': 'ASTER_Thermal_IR'  # Specify your folder in Google Drive
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

# Export each image in the collection to Google Drive
aster_with_thermal_ir_list = aster_with_thermal_ir.toList(aster_with_thermal_ir.size())
for i in range(aster_with_thermal_ir.size().getInfo()):
    image = ee.Image(aster_with_thermal_ir_list.get(i))
    exportImageToDrive(image)

print("Export tasks started. Check Google Drive for export status.")